### **一、码书模型（Code Book）**

#### **核心思想**

码书模型通过为**每个像素建立动态码本**来描述背景的复杂变化。每个码本包含多个码元（阈值范围），通过匹配当前像素值与历史码元来判断其属于背景还是前景。

- **码元更新**：若像素值匹配码元，则更新码元的阈值；若不匹配，则新增码元以应对动态背景（如树叶晃动、光照渐变）。
- **适应性**：适合处理多模态背景（如水面波纹、动态植被），但对剧烈变化的场景（如突然光照突变）鲁棒性较弱。

#### **技术分类**

- **背景差分法**：属于基于像素的背景建模方法，通过建立背景模型并与当前帧对比检测前景。

In [ ]:
# 代码实现（hsv空间）
import cv2
import numpy as np

# 初始化视频捕捉
cap = cv2.VideoCapture('video.mp4')

# 读取第一帧
ret, frame = cap.read()
hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

# 初始化码书
codebook = np.zeros_like(hsv, dtype=np.float32)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # 更新码书
    codebook = cv2.addWeighted(codebook, 0.9, hsv, 0.1, 0)
    
    # 计算前景掩码
    fg_mask = cv2.absdiff(hsv, codebook)
    fg_mask = cv2.cvtColor(fg_mask, cv2.COLOR_BGR2GRAY)
    _, fg_mask = cv2.threshold(fg_mask, 50, 255, cv2.THRESH_BINARY)
    
    # 显示结果
    cv2.imshow('Foreground Mask', fg_mask)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
